# Inżynieria lingwistyczna
Ten notebook jest oceniany półautomatycznie. Nie twórz ani nie usuwaj komórek - struktura notebooka musi zostać zachowana. Odpowiedź wypełnij tam gdzie jest na to wskazane miejsce - odpowiedzi w innych miejscach nie będą sprawdzane (nie są widoczne dla sprawdzającego w systemie).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

---

# Zadanie domowe 2

In [6]:
import pandas as pd
import numpy as np
       

## Zadanie 1 - eksploracja przestrzeni zagnieżdżeń
Wczytajmy do przestrzeni plik zagnieżdżeń, który należy pobrać ze strony:
https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.pl.vec Są to zagnieżdzenia dla języka polskiego uzyskane systemem fastText.

Do przestrzeni wczytujemy tylko 100 tys. najczęstrzych słów, tak aby operacje przebiegały szybciej.

In [8]:
import io
def load_vectors(fname, limit = 100000):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    n = min(n,limit)
    embeddings = np.empty((n,d), dtype = np.float)
    words_idx = []
    for i, line in enumerate(fin):
        if i >= limit:
            break
        tokens = line.rstrip().split(' ')
        words_idx.append(tokens[0])
        embeddings[i] =  np.array(tokens[1:]).astype(np.float)
    return words_idx, embeddings
words_idx, embeddings = load_vectors('wiki.pl.vec')

Poniższe zadania mają na celu poekserymentowanie z przestrzenią zagnieżdżeń, ale też zrozumienie stojącymi za nich operacji. Dozwolone jest korzystanie tylko z podstawowych operatorów Python i numpy (w szczególności zakaz dotyczy: sklearn, gensim, fasttext itd.)

Jeśli potrzebujesz do dalszego przetwarzania przeprowadzenia jakichś normalizacji macierzy -- możesz wstępnie przetworzyć macierz zagnieżdzeń poniżej. Pamiętaj, że sprawdzarka będzie używała wywołań do `embeddings` (i `words_idx`) -- musisz nadpisać macierz zagnieżdzeń. To pole jest pomocnicze i nie podlega ocenie.

In [10]:
#!wget -q -O wiki.pl.vec https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.pl.vec

# This will map words into indexes to speed up search index
words_idx = dict([(word, index) for index, word in enumerate(words_idx)])

Zaimplementuj funkcję, która obliczy podobieństwo kosinusowe pomiędzy dwoma wyrazami.

In [11]:
def calc_sim(word_a, word_b, words_idx, embeddings):
    """
    Calculate cosine similarity between two words with using embeddings and words idx lists.
    Known issues: If not found word - throw exception
    """
    # Get from embeddings vector assigned to word A
    a = embeddings[words_idx[word_a]]
    
    # Get from embeddings vector assigned to word B
    b = embeddings[words_idx[word_b]]
    
    # Cosine similarity
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [12]:
from nose.tools import assert_almost_equal
assert_almost_equal(calc_sim("bieber", "rihanna", words_idx, embeddings), calc_sim("rihanna", "bieber", words_idx, embeddings))

Policz podobieństwo pomiędzy wyrazem `bieber` a wyrazami:
    - `rihanna`
    - `piłsudski`
    - `kanada`
    - `polska`
    - `piosenka`

In [13]:
calc_sim('bieber', 'rihanna', words_idx, embeddings)

0.524583248263655

In [14]:
print(calc_sim('bieber', 'rihanna', words_idx, embeddings))
print(calc_sim('bieber', 'piłsudski', words_idx, embeddings))
print(calc_sim('bieber', 'kanada', words_idx, embeddings))
print(calc_sim('bieber', 'polska', words_idx, embeddings))
print(calc_sim('bieber', 'piosenka', words_idx, embeddings))

0.524583248263655
0.1930395805146356
0.20042742126487934
0.12505934735679372
0.2874871368858332


Zaimplementuj funkcję, która zwróci najbardziej podobne słowa (miara kosinusowa) do danego słowa `word`. W wyniku wypisz tylko `top` słów z najbliższymi zagnieżdżeniami, pomijając słowo `word`.

In [15]:
# Used itemgetter to speed-up sorting elements in list
from operator import itemgetter

def find_similar(word, words_idx, embedding_matrix, top=10):
    """
    Build similarity list between `word` and each word.
    Select `top` number of elements and return it.
    Ignore `word` on this list
    """
    # Calculate similarity between `word` and each element in dictionary
    similarity = [(calc_sim(word, w, words_idx, embedding_matrix), w) for w in words_idx.keys() if w != word]
    
    # Order by similarity - highest value at first
    similarity = sorted(similarity, key=itemgetter(0), reverse=True)
    
    # Return `top` number of similar words
    return [w for (sim, w) in similarity[0:top]]

In [16]:
assert len(find_similar("radość", words_idx, embeddings)) == 10

Znajdź najbardziej podobne słowa do kobieta, politechnika, mateusz, szczecin, niemcy, piłsudski

In [17]:
find_similar('kobieta', words_idx, embeddings)

['kobietą',
 'dziewczyna',
 'mężczyzna',
 'kobietę',
 'dziewczynka',
 'mężczyznę',
 'staruszka',
 'mężczyzną',
 'kobiecie',
 'mężczyzny']

In [18]:
print(find_similar('kobieta', words_idx, embeddings))
print(find_similar('politechnika', words_idx, embeddings))
print(find_similar('mateusz', words_idx, embeddings))
print(find_similar('szczecin', words_idx, embeddings))
print(find_similar('niemcy', words_idx, embeddings))
print(find_similar('piłsudski', words_idx, embeddings))

['kobietą', 'dziewczyna', 'mężczyzna', 'kobietę', 'dziewczynka', 'mężczyznę', 'staruszka', 'mężczyzną', 'kobiecie', 'mężczyzny']
['politechniki', 'politechniką', 'politechnikę', 'politechniczny', 'politechnice', 'politechnicznej', 'politechnicznego', 'politechnicznym', 'inżynierska', 'elektrotechnika']
['łukasz', 'bartłomiej', 'bartosz', 'kacper', 'marcin', 'mateusza', 'tomasz', 'patryk', 'rafał', 'mateuszem']
['szczecinek', 'szczeciński', 'szczecinem', 'gryfino', 'szczecinie', 'stargard', 'szczecina', 'koszalin', 'szczecińska', 'świnoujście']
['niemieccy', 'naziści', 'alianci', 'okupanci', 'polacy', 'hitlerowcy', 'niemieckie', 'rosjanie', 'niemców', 'niemcom']
['piłsudskim', 'piłsudskiego', 'piłsudskiemu', 'sosnkowski', 'mościcki', 'śmigły', 'józef', 'żeligowski', 'piłsudczyków', 'sosnkowskiego']


Krótko skomentuj wyniki dla słowa `niemcy`. Które z powstałych analogii biorą się z semantycznego powiązania a które z semantycznego podobieństwa?

In [20]:
# Komentarz:  Część otrzymanych analogii pochodzi z nawiązania do II WŚ i ataku hitlerowskich Niemiec na Polskę.
#             Mamy także analogie z semantycznego podobieństwa.
# Semantycznie powiązane: 'alianci', 'polacy', 'rosjanie', 'naziści', 'okupanci', 'hitlerowcy'
# Semantyczne podobieństwo: 'niemieccy', 'niemieckie', 'niemców', 'niemcom'

Zaimplementuj funkcje szukającą brakującego elementu relacji ,,`word_a` jest do `word_a2` jak `word_b` jest do...''. Funkcja powinna zwrócić 10 najbardziej pasujących słow z pominięciem słów będących jej argumentami.

In [21]:
def find_similar_pair(word_a, word_a2, word_b, words_idx, matrix, top=10):
    """
    Find similar pair - missing `wordB*` in `wordA` is similar to `wordA*` like `wordB` to `wordB*`
    """
    # Not allowed words
    not_allowed_words = set([word_a, word_a2, word_b])
    
    # Calculate similarity based on formula from lecture
    similarity = [(calc_sim(w, word_b, words_idx, matrix) - calc_sim(w, word_a, words_idx, matrix) + calc_sim(w, word_a2, words_idx, matrix), w) for w in words_idx.keys() if w not in not_allowed_words]
    
    # Sort similarity - first element with highest value
    similarity = sorted(similarity, key=itemgetter(0), reverse=True)
    
    # Return `top` number of similar words
    return [w for (sim, w) in similarity[0:top]]

In [22]:
assert find_similar_pair( "mężczyzna", "król", "kobieta", words_idx, embeddings)[0] == "królowa"

Pieniądze są do profesora jak wiedza do...

In [23]:
find_similar_pair('pieniądze', 'profesora', 'wiedza', words_idx, embeddings, top=1)[0]

'habilitacja'

Mateusza jest do mateusz jak łukasza do ...

In [24]:
find_similar_pair('mateusza', 'mateusz', 'łukasza', words_idx, embeddings, top=1)[0]

'łukasz'

Warszawa jest do "polska" jak "berlin" do ...

In [25]:
find_similar_pair('warszawa', 'polska', 'berlin', words_idx, embeddings, top=1)[0]

'niemiecka'

Zurich jest do ETH jak Poznań do ...

In [26]:
find_similar_pair( "zurich", "eth", "poznań", words_idx, embeddings)

['„poznań',
 'wrocław',
 'poznania',
 'poznańskie',
 'gniezno',
 'kraków',
 'poznaniu',
 'uam',
 'wielkopolski',
 'poznańską']

Niemcy są do Merkel jak Polska do ...

In [27]:
find_similar_pair( "niemcy", "merkel", "polska", words_idx, embeddings)

['kaczyńska',
 'ekonomistka',
 'lewandowska',
 'kwaśniewska',
 'lekarka',
 'parlamentarzystka',
 'marcinkiewicz',
 'nowacka',
 'dziennikarka',
 'olszewska']

Na wektorach możemy wykonywać standardowe operacje algebry liniowej takie jak np. projekcja czyli rzutowanie danych na jakichś zbiór osi (więcej: notatki z algebry liniowej np. https://ocw.mit.edu/courses/mathematics/18-06sc-linear-algebra-fall-2011/least-squares-determinants-and-eigenvalues/projections-onto-subspaces/). W szczególności może to się przydać do zrzutowania słowa na przestrzeń w której pewny wybrany kierunek (wskazywany przez wektor) jest eliminowany.

Do czego może to się przydać? Jeśli uruchomisz funkcję `find_similar` dla słowa ,,mateusza'' znajdziesz m.in. ,,łukasza'' ale także ,,ewangelia'', ,,ewangelisty'' i ,,apostoła''. Chcąc pominąc kontekst religijny tego słowa możesz zrzutować jego reprezentacje na przestrzeń bez wektora ,,ewangelia'' i poszukać jego najbliższych sąsiadów (którymi będą teraz po prostu imiona męskie). Zaimplementuj taką funkcję.


In [ ]:
def find_similar_with_rejection(word, remove, words_idx, matrix, top=10):
    """
    Działanie analogiczne do find_similar z dodatkowym parametrem remove, 
    który jest *listą* słów, które należy wyrzucić poprzez projekcję.
    Dla remove=[] powinno się zwracać dokładnie to samo co find_similar
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    
print('Standardowe poszukiwanie:', find_similar_with_rejection('mateusza', [] , words_idx, embeddings))
print('Poszukiwanie po projekcji:', find_similar_with_rejection('mateusza', ['ewangelia'] , words_idx, embeddings))

In [ ]:
assert "ewangelii" in find_similar_with_rejection("mateusza",[] , words_idx, embeddings)
assert "ewangelii" not in find_similar_with_rejection("mateusza",["ewangelia"] , words_idx, embeddings)
assert "ewangelisty" not in find_similar_with_rejection("mateusza",["ewangelia"] , words_idx, embeddings)


Analogicznie słowo ,,java'' jest nie tylko nazwą języka programownia (https://pl.wikipedia.org/wiki/Java_(ujednoznacznienie)) -- jest np. nazwą geograficzną (indonezyjska wyspa koło Sumatry). Sprawdź jakie wyrazy są podobne do "java" oraz po odrzuceniu kierunku "javascript" (tj. kierunku związanego z językami programowania).

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Spróbuj poekseprymentować samemu!

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Wykonanie projekcji w przestrzeni zagnieżdżeń może być jedną z prostych technik zwalczenia tzw. gender bias (http://wordbias.umiacs.umd.edu/) w reprezentacji słów. Okazuje się, że wykonanie projekcji macierzy zagnieżdżeń na przestrzeń w której ,,brakuje kierunku he-she'' może być bardzo prostą techniką zredukowania tego typu obciążenia.

## Zadanie 2 - zagnieżdżenia dokumentów
W tym ćwiczeniu powócimy do zbioru tweetów, który analizowaliśmy w poprzednim dokumencie.

In [28]:
from helpers import DataSet
training_set = DataSet(['tweets.txt'])

Reading data set ['tweets.txt']


In [29]:
for i in training_set.tweets:
    print(i.text)
    print(i.tokens)
    print(i.clazz)
    break

dear @Microsoft the newOoffice for Mac is great and all, but no Lync update? C'mon.
['dear', '@microsoft', 'the', 'newooffice', 'for', 'mac', 'is', 'great', 'and', 'all', ',', 'but', 'no', 'lync', 'update', '?', "c'mon", '.']
negative


Tym razem do zbudowania reprezentacji będziemy używać narzędzie Universal Sentence Encoder stworzone przez Googla na bazie głębokiej sieci uśredniającej (i architektur rekurencyjnych). Poniższy kod pokazuje sposób użycia tego narzędzia. 
Kod spokojnie można wywoływać na CPU -- choć ściąganie modelu trochę może potrwać.

In [39]:
# !wget -q -O universal-sentence-encoder-4.tar.gz https://tfhub.dev/google/universal-sentence-encoder/4?tf-hub-format=compressed
# !tar -xzf universal-sentence-encoder-4.tar.gz
import tensorflow as tf
import tensorflow_hub as hub
embed = hub.load('universal-sentence-encoder-4')
embeddings = embed([
    "The quick brown fox jumps over the lazy dog.",
    "I am a sentence for which I would like to get its embedding"])
print (embeddings)

tf.Tensor(
[[-0.03133018 -0.06338634 -0.01607502 ... -0.03242778 -0.04575741
   0.05370456]
 [ 0.05080861 -0.01652428  0.01573781 ...  0.00976659  0.03170123
   0.0178812 ]], shape=(2, 512), dtype=float32)


Wykorzystując reprezetnację USE wytrenuj wybrany klasyfikator z pakietu `sklearn` i zweryfikuj jego jakość działania.

In [44]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Split test and train dataset
X = list(embed([tweet.text for tweet in training_set.tweets]))
y = [tweet.clazz for tweet in training_set.tweets]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=127228)

clf = SVC()

# Fit classifier
clf.fit(X_train, y_train)

# Check prediction score
accuracy_score(y_test, clf.predict(X_test))

0.6542124542124542

Skomentuj wyniki i odnieś je do wyników z poprzedniego zadania domowego. Na ile użycie reprezentacji rozproszonych pozwoliło na poprawę wyników?

In [45]:
# Wyniki:
# * FeatureHasher - 0.5296703296703297
# * Bag of clusters - 0.5882783882783883
# * Universal Sentence Encoder - 0.6542124542124542
# 
# Dzięki wykorzystaniu USE udało się poprawić wyniki.
# Wykorzystuje on embeddingi do wyrażenia sentencji w postaci reprezentacji wektorowej.
# Najbardziej jest to widoczne w odniesieniu do bazowej wersji z Feature Hasherem, gdzie obserujemy poprawę o 13 pkt procentowych.

# Zadanie 3 - konstruowanie zagnieżdżeń
W tym ćwiczeniu kontynuujemy pracę z tweetami, ale pomijamy całkowicie ich klasy. Zbiór tweetów potraktujemy jako korpus do nauczenia zagnieżdżeń słów przy pomocy macierzy PMI.
- Wypełnij macierz kontekst - dokument przy użyciu symetrycznego okna o promieniu 4 (po 4 słowa w każdą stronę)
- Możesz ograniczyć słownictwo do 10K słów
- Przekształć macierz w macierz PPMI
- Stwórz zagnieżdżenia wykorzystując dekompozycję SVD do wybranej wymiarowości $d$ (ze względu na koszt obliczeniowy może to być mała wymiarowość np. $d=10$)

In [74]:
from itertools import chain

def hyperspace_analogue_to_language(tweets, r=4):
    # Key - `word`, Values - dict with `another_word` => `count`
    output = dict()
    
    # Prepare generator with weight assigned to each position in window
    weights = [0, *list(range(r, 0, -1))]

    for tweet in training_set.tweets:
        sentence = tweet.tokens
        for index, center_word in enumerate(sentence):
            # Get row assigned to word
            row = output.get(center_word, {})

            # Iterate over elements in window and combined weights
            window = iter(sentence[max(index - r, 0):index + r + 1])
            for word, weight in zip(window, weights):
                row[word] = row.get(word, 0) + weight

            # Store updated row in dictorionary
            output[center_word] = row

            # If index < window size - add at beggining of weights list new weight
            if index < r:
                weights.insert(0, r - index)
    
    return output

def transform_into_ppmi_matrix():
    
    
matrix_dict = hyperspace_analogue_to_language(training_set.tweets[0:100])

{'dear': {'dear': 9,
  '@microsoft': 4,
  'the': 3,
  'newooffice': 2,
  'for': 1,
  'german': 2,
  'citizens': 3,
  'any': 4,
  'concerns': 1,
  'friends': 2,
  'new': 3,
  'york': 4,
  'is': 1,
  '@dlepeska': 3,
  '@timepdc': 4,
  'david': 2,
  ',': 13,
  'i': 11,
  'hope': 1,
  '@burngorman': 4,
  'burn': 2,
  'just': 4,
  'saw': 1,
  'ppl': 2,
  'who': 3,
  'attribute': 4,
  'vocal': 1,
  'guide': 1,
  'my': 2,
  'life': 3,
  'lord': 2,
  'do': 3,
  "n't": 4,
  'let': 1,
  'journalists': 2,
  'please': 4,
  'quit': 1,
  'god': 2,
  'may': 4,
  'we': 1,
  'all': 2,
  'people': 3,
  'know': 1},
 '@microsoft': {'dear': 4,
  '@microsoft': 94,
  'the': 36,
  'newooffice': 3,
  'for': 22,
  'mac': 1,
  'how': 6,
  'about': 4,
  'you': 27,
  'make': 2,
  'in': 7,
  'november': 2,
  '?': 5,
  '@majornelson': 4,
  'operating': 1,
  'system': 2,
  '.': 38,
  '#windows10': 8,
  '@mikewolf1980': 4,
  'i': 21,
  'will': 9,
  'be': 12,
  'downgrading': 1,
  '2nd': 10,
  'computer': 3,
  'with': 

Przetestuj działanie Twoich zagnieżdżeń wykorzystując funkcję `find_similar` na wybranych słowach.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()